In [1]:
!pip install transformers
!pip install tensorflow-gpu
!pip install tf-keras
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
import numpy as np
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, set_seed
from tensorflow.keras.optimizers import Adam

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        

2024-04-30 16:31:53.316272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
set_seed(123)
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token 

In [3]:
model = TFGPT2LMHeadModel.from_pretrained("distilgpt2")

2024-04-30 16:31:55.332337: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-30 16:31:55.335874: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-30 16:31:55.335901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-30 16:31:55.337357: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-30 16:31:55.337384: I external/local_xla/xla/stream_executor

In [4]:
input_sequences = []
target_sequences = []

with open("dataset.txt", "r", encoding="utf-8") as file:
    for line in file:
        question, answer = line.strip().split('\t')
        combined_text = question + tokenizer.eos_token + answer
        encoded = tokenizer.encode_plus(combined_text, max_length=64, truncation=True, padding="max_length", return_tensors="tf")
        input_sequences.append(encoded['input_ids'][0])
        target_sequences.append(tf.concat([encoded['input_ids'][0][1:], [tokenizer.pad_token_id]], axis=0))


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))
dataset = dataset.shuffle(1024).batch(32)

In [6]:
tf.keras.optimizers.Adam.learning_rate = 0.001
tf.keras.optimizers.Adam.beta_1 = 0.9
tf.keras.optimizers.Adam.beta_2 = 0.999
tf.keras.optimizers.Adam.epsilon = 1e-07
tf.keras.optimizers.Adam.amsgrad = False

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [8]:
model.fit(dataset, epochs=10)

Epoch 1/10
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1714494747.470839    7373 service.cc:145] XLA service 0x7fe5c5648430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714494747.470882    7373 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-04-30 16:32:27.475791: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-30 16:32:27.489080: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1714494747.531707    7373 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


117/117 [==============================] - 41s 104ms/step - loss: 1.0250
Epoch 2/10
117/117 [==============================] - 13s 109ms/step - loss: 0.5924
Epoch 3/10
117/117 [==============================] - 13s 112ms/step - loss: 0.4388
Epoch 4/10
117/117 [==============================] - 13s 112ms/step - loss: 0.3519
Epoch 5/10
117/117 [==============================] - 13s 107ms/step - loss: 0.2918
Epoch 6/10
117/117 [==============================] - 12s 104ms/step - loss: 0.2497
Epoch 7/10
117/117 [==============================] - 12s 104ms/step - loss: 0.2193
Epoch 8/10
117/117 [==============================] - 12s 104ms/step - loss: 0.1948
Epoch 9/10
117/117 [==============================] - 12s 104ms/step - loss: 0.1721
Epoch 10/10
117/117 [==============================] - 13s 108ms/step - loss: 0.1568


In [9]:
model.save_pretrained('fine_tuned_gpt2')

In [10]:
import tensorflow as tf
import numpy as np
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, set_seed
from tensorflow.keras.optimizers import Adam
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token 
model = TFGPT2LMHeadModel.from_pretrained('fine_tuned_gpt2')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at fine_tuned_gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [11]:
import tensorflow as tf

def generate_text(prompt, max_length=50, temperature=0.7, top_k=50):
    seed = np.random.randint(0, 10000)
    tf.random.set_seed(seed)
    
    prompt = prompt + tokenizer.pad_token
    input_len = len(prompt)
    generated_text = prompt 
    input_ids = tokenizer.encode(prompt, return_tensors="tf")
    
    for _ in range(max_length):
        outputs = model(input_ids)
        predictions = outputs.logits[:, -1, :]

        predictions = predictions / temperature

        top_k_values, top_k_indices = tf.nn.top_k(predictions, k=top_k)
        values = tf.nn.softmax(top_k_values, axis=-1)
        sampled_token_index = tf.random.categorical(tf.math.log(values), num_samples=1)
        sampled_token_id = tf.gather(top_k_indices, sampled_token_index, batch_dims=1).numpy().squeeze()

        if sampled_token_id == tokenizer.eos_token_id:
            break

        generated_text += tokenizer.decode([sampled_token_id])
        input_ids = tf.concat([input_ids, [[sampled_token_id]]], axis=-1)
        
    return generated_text [input_len:]

In [13]:
prompt = "where can I find the best food around here?"

for i in range(10):
    generated_text = generate_text(prompt, max_length=100)
    print("Generated text:", generated_text)

Generated text: i used all my savings on this one.
Generated text: i don't know. look in the fridge.
Generated text: i don't know.
Generated text: i don't know. maybe prices are cheaper than police?
Generated text: i know that.
Generated text: i bought it from the macy's at the santa anita mall.
Generated text: i know, i don't want to eat there.
Generated text: i don't know.
Generated text: i filled up on the bread.
Generated text: i know, i don't like to eat leftovers.
